## 研究の流れ

この ipynb と README.md を見るだけで流れがわかるようにしました。  
なお、各処理の詳細が知りたい場合は各ファイル、Docstring, [README.md](./README.md), [Notion](https://vizlabstudent.notion.site/de778517ea47444c9598d1f5147d78da?v=9dd0c88c9540426db2fa5a4308baf536&pvs=4)を閲覧してください。


In [ ]:
# 標準モジュールのインポート
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import subprocess
from glob import glob

# src配下のファイルを読み込めるようにする
sys.path.append(os.getcwd() + "/src")

In [ ]:
# 共通する自作モジュールのインポート
from Visualization.SnapData import SnapData


In [ ]:
# ディレクトリの作成と定義
from config.params import ROOT_DIR, SNAP_PATH, SRC_PATH, IMAGE_PATH, ML_DATA_DIR, ML_RESULT_DIR, datasets, variable_parameters

data_dir = ROOT_DIR + "/data"

# 出力先フォルダの作成
subprocess.run([ROOT_DIR + "/etc/BasePath.bat"])


### 1. データの加工


##### 1.1 元データを各種パラメータに分割する


In [ ]:
import os
import sys
import shutil
import subprocess
from glob import glob

sys.path.append(os.getcwd())

from config.params import BIN_PATH, ROOT_DIR, SNAP_PATH, SRC_PATH, datasets
from Processing.separatorLoop import move_file, rename_file, set_ij

In [ ]:
# パラメータの定義
items1 = ["density", "enstrophy", "pressure", "magfieldx", "magfieldy", "magfieldz", "velocityx", "velocityy", "velocityz"]
items2 = ["magfield1", "magfield2", "magfield3", "velocity1", "velocity2", "velocity3"]
xyz = {1: "x", 2: "y", 3: "z"}

for dataset in datasets:
    ij = set_ij(dataset)
    if ij:
        i, j = ij
    else:
        print("Value Error", "入力したデータセットは使用できません")
        sys.exit()

    # bat ファイルの実行
    # 基本的に加工したデータの保存先のフォルダの作成
    print("MAKE", "ディレクトリの作成")
    subprocess.run([BIN_PATH + "/Snaps.bat", str(dataset)])

    # ログの保存先
    files = glob(ROOT_DIR + f"/data/ICh.dataset=50.ares=1.0d-{i}.adiffArt=1.0d-{j}.h00.g00.BCv1=0.0/Snapshots/*")
    for file in files:
        # 元データの分割処理の実行
        subprocess.run([SRC_PATH + "/Processing/cln/separator.exe", f"{file}"])
        _, _, _, param, job = map(lambda x: int(x) if x.isnumeric() else x, os.path.basename(file).split("."))
        print("OPEN", f"File snap{i}{j}.{param:02d}.{job:02d}")

        # 出力されたファイル名の変更
        for item2 in items2:
            if os.path.exists(item2):
                rename_file(xyz, param, job, item2)

            else:  # 見つからない場合
                print("NotFound", f"ファイル {item2}.{param:02d}.{job:02d}")

        # 出力されたファイルの移動
        for item1 in items1:
            if os.path.exists(item1):
                move_file(dataset, item1)

            else:  # 見つからない場合
                print("NotFound", f"ファイル {item2}.{param:02d}.{job:02d}")

        print("CLOSE", f"File {item2}.{param:02d}.{job:02d}")

    # coordn を最後に移動させる
    for i in range(1, 4):
        shutil.move("coord" + xyz[i], SNAP_PATH + f"/snap{dataset}")

    print("END", "処理終了")


##### 1.2 バイナリを .npy に変換


In [ ]:
import os
import sys
from glob import glob

sys.path.append(os.getcwd() + "/src")

from config.params import SNAP_PATH, datasets, variable_parameters, set_dataset
from Processing.snap2npy import snap2npy

In [ ]:
dataset = set_dataset(input())

sp = SnapData()
print("START", f"Snap{dataset} 開始")

for param in variable_parameters:
    print("START", f"{param} 開始")

    for path in glob(SNAP_PATH + f"/snap{dataset}/{param}/*/*"):
        # print(path)
        snap2npy(sp, path, dataset)

    print("END", f"{param} 終了")
print("END", f"Snap{dataset} 終了")


## 2. 可視化


#### 2.1 各種可視化


In [ ]:
import os
import sys
from glob import glob

sys.path.append(os.getcwd() + "/src")

from config.params import SNAP_PATH, datasets
from Visualization.Visualize import VisualizeMethod

In [ ]:
dataset = set_dataset(input())

target_path = SNAP_PATH + f"/snap{dataset}"
viz = VisualizeMethod(dataset)

files = {}
files["density"] = glob(target_path + f"/density/*/*")
files["velocityx"] = glob(target_path + f"/velocityx/*/*")
files["velocityy"] = glob(target_path + f"/velocityy/*/*")
for dens_path, vx_path, vy_path in zip(files["density"], files["velocityx"], files["velocityy"]):
    viz.drawEnergy_for_velocity(dens_path, vx_path, vy_path)

files["magfieldx"] = glob(target_path + f"/magfieldx/*/*")
files["magfieldy"] = glob(target_path + f"/magfieldy/*/*")
for magx_path, magy_path in zip(files["magfieldx"], files["magfieldy"]):
    viz.drawEnergy_for_magfield(magx_path, magy_path)

files["enstrophy"] = glob(target_path + f"/enstrophy/*/*")
for target in ["velocityx", "velocityy", "magfieldx", "magfieldy", "density", "enstrophy"]:
    for path in files[target]:
        viz.drawHeatmap(path)
        viz.drawEdge(path)

#### 2.2 AVS


#### 2.3 StreamLines


#### 2.4 LIC


In [ ]:
import os
import sys
from glob import glob

sys.path.append(os.getcwd() + "/src")

from config.params import SNAP_PATH, IMAGE_PATH, datasets
from LIC.LIC import LicMethod

In [ ]:
# ログ取得の開始
print("START", "処理開始\n\n")

dataset = set_dataset(input())
size = "left" # right

print("START", f"{dataset}.{size.split('_')[1]} 開始\n")
lic = LicMethod()

# 入出力用path の作成
in_dir = SNAP_PATH + f"/{size}/snap{dataset}"
dir_basename = os.path.basename(in_dir)  # snap77
out_dir = IMAGE_PATH + "/LIC"
base_out_path = out_dir + "/" + os.path.basename(in_dir) + "/" + size.split("_")[1]  # ./IMAGES/LIC/snap77/left
lic.makedir(f"/LIC/snap{dataset}/{size.split('_')[1]}")

# バイナリファイルの取得
binary_paths = glob(in_dir + "/magfieldx/*/*.npy")

# ファイルが無い場合
if binary_paths == []:
    print("ERROR", "File not Found\n")
    sys.exit()

for xfile in binary_paths:
    print("START", f"{os.path.splitext(os.path.basename(xfile))[0]} 開始\n")
    yfile = xfile.replace("magfieldx", "magfieldy")
    file_name = os.path.splitext(os.path.basename(xfile.replace("magfieldx", "magfield")))
    out_path = base_out_path + f"/lic_{dir_basename}.{os.path.basename(base_out_path)}.{file_name[0]}.bmp"
    # print(out_path) # ./IMAGES/LIC/snap77/lic_snap77.magfieldx.01.14.bmp

    if not os.path.exists(out_path):
        # 引数の作成
        props = lic.set_command(xfile, yfile, out_path)
        # 実行 (1画像20分程度)
        lic.LIC(props)

        # temp ファイルの削除
        lic.delete_tempfile(props[1], props[2])

    print("END", f"{os.path.splitext(os.path.basename(xfile))[0]} 終了\n")

print("END", f"{dataset} 終了\n")

### 3. 教師データの作成


#### 3-1 ビューワの作成


In [1]:
import os
import sys

sys.path.append(os.getcwd() + "/src")

from Processing.viewer.createViewer import _sort_paths
from config.params import IMAGE_PATH, SRC_PATH

In [2]:
for size in ["left", "right"]:
    # paths = _sort_paths(paths) # snapの命名規則をもとに時系列順に並び変える。
    paths = glob(IMAGE_PATH + f"/LIC/snap{dataset}/{size}/*.bmp")
    paths_sorted = _sort_paths(paths)

    # viewer用のファイル列を作成する
    path_list_str = "\n"
    for path in paths_sorted:
        path_str = path.replace("\\", "/")
        path_list_str += f"\t\t\t'{path_str}', \n"

    # html の読み込み
    with open(SRC_PATH + "/Processing/viewer/template/viewer_template.html", "r", encoding="utf-8") as f:
        html = f.read()

    # 可視化した.bmpのpathの一覧をhtml に追記
    html = html.replace("{ replaceblock }", path_list_str)

    # html の保存
    out_name = SRC_PATH + f"/Processing/viewer/template/lic_viewer{dataset}.{size}.html"
    with open(out_name, "w", encoding="utf8") as f:
        f.write(html)

#### 3-2 画像の分割

`python ./src/Processing/viewer/writer.py`  
を実行し、  
`./src/Processing/viewer/template/lic_viewer77.html`
を Web で開く (Drug & Drop)


#### 3.3 データの切り取り


In [ ]:
import os
import sys

sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + "/src")

from config.params import set_dataset
from Processing.fusion_npy import CrateTrain

In [ ]:
dataset = set_dataset(input())

md = CrateTrain()
for val in ["magfieldx", "magfieldy", "velocityx", "velocityy", "density"]:
    md.cut_and_save(dataset, val)


#### 3.4. データの合成

In [ ]:
import os
import sys
from glob import glob

sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + "/src")
from config.params import ML_DATA_DIR, labels, set_dataset
from Processing.fusion_npy import CrateTrain

In [ ]:
dataset = set_dataset(input())

md = CrateTrain()
props_params = [
    (["magfieldx", "magfieldy"], "mag_tupledxy", md.kernellistxy),
    (["velocityx", "velocityy", "density"], "energy", md.kernelEnergy),
]
OUT_DIR = ML_DATA_DIR + f"/snap{dataset}"

# /images/0131_not/density/density_49.50.8_9.528
for val_params, out_basename, kernel in props_params:
    for label in labels:
        npys = OUT_DIR + f"/point_{label}"

        for img_path in glob(npys + "/" + val_params[0] + "/*.npy"):
            im_list = md.loadBinaryData(img_path, val_params)  # 混合データのロード
            resim = kernel(*im_list)  # データの作成

            # 保存先のパスの作成
            # /MLdata/snap{dataset}/{out_basename}/{out_basename}_{dataset}.{param}.{job}_{centerx}.{centery}.npy
            # /MLdata/snap77/energy/energy_77.01.03_131.543.npy
            out_path = npys + "/" + out_basename + "/" + os.path.basename(img_path).replace(val_params[0], out_basename)
            md.saveFusionData(resim, out_path)  # データの保存


### 4. 機械学習


#### 4.1 k-Means


In [ ]:
import os
import sys
import numpy as np
from glob import glob

sys.path.append(os.getcwd() + "\src")

from config.params import SNAP_PATH, datasets, variable_parameters
from MachineLearning.KMeans import ClusteringMethod

In [ ]:
cluster = ClusteringMethod()

for dataset in datasets:
    for target in variable_parameters:
        path_list = glob(SNAP_PATH + f"/snap{dataset}/{target}/*/*")
        num_of_data = len(path_list)  # リコネクションがない画像の枚数

        temp_data = cluster.compress(cluster.loadSnapData(path_list[0], z=3))
        IMGSHAPE = temp_data.shape

        N_col = IMGSHAPE[0] * IMGSHAPE[1] * 1  # 行列の列数
        X_train = np.zeros((num_of_data, N_col))  # 学習データ格納のためゼロ行列生成
        y_train = np.zeros((num_of_data))  # 学習データに対するラベルを格納するためのゼロ行列生成

        # リコネクションがない画像を行列に読み込む
        for idx, item in enumerate(path_list[:10]):
            X_train[idx, :] = cluster.load_regularize(item)
            y_train[idx] = 0  # リコネクションがないことを表すラベル

        X_train_pca = cluster.PCA(X_train)
        cluster_labels = cluster.KMeans(X_train_pca)
        df_re = cluster.save_result(cluster_labels, path_list, dataset)
        # display(df_re)

#### 4.2 SVM


#### 4.3 非線形 SVM


#### 4.4 k 近傍法


#### 4.5 XGBoost
